<a href="https://colab.research.google.com/github/PraveenKumarTV/NitIntern_nlp/blob/main/ImageTranslator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

mixed_precision.set_global_policy('mixed_float16')

img_size = (64, 64)
data_path = '/content/drive/MyDrive/Colab Notebooks/TAMIL NLP/datasets/brami'
num_classes = 27
batch_size = 64
epochs = 20

def load_data():
    X, y = [], []
    for label in range(num_classes):
        folder_path = os.path.join(data_path, str(label))
        for file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_size)
            X.append(img)
            y.append(label)
    X = np.array(X, dtype=np.float32) / 255.0
    X = X.reshape(-1, img_size[0], img_size[1], 1)
    y = to_categorical(y, num_classes)
    return X, y

X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

model = Sequential([
    Input(shape=(img_size[0], img_size[1], 1)),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax', dtype='float32')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, epochs=epochs, validation_data=test_ds)

model.save("brahmi_to_tamil_model.h5")


Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 11s 234ms/step - accuracy: 0.1082 - loss: 3.2355 - val_accuracy: 0.2462 - val_loss: 2.8469
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2342 - loss: 2.7652 - val_accuracy: 0.4474 - val_loss: 2.0278
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4490 - loss: 1.9511 - val_accuracy: 0.6126 - val_loss: 1.4998
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5953 - loss: 1.4295 - val_accuracy: 0.6967 - val_loss: 1.0782
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7055 - loss: 1.0144 - val_accuracy: 0.7658 - val_loss: 0.9158
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7660 - loss: 0.7891 - val_accuracy: 0.8168 - val_loss: 0.7247
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8136 - loss: 0.6497 - val_accuracy: 0.8018 - val_loss: 0.7057
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8470 - loss: 0.4791 - val_accuracy: 0.7988 - val_lo

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("brahmi_to_tamil_model.h5")

# Image configuration
img_size = (64, 64)

# Label map (27 characters)
label_map = {
    0: 'அ', 1: 'ஆ', 2: 'இ', 3: 'ஈ', 4: 'உ', 5: 'ஊ', 6: 'எ', 7: 'ஏ', 8: 'ஐ',
    9: 'ஒ', 10: 'ஓ', 11: 'ஔ', 12: 'க', 13: 'ங', 14: 'ச', 15: 'ஞ',
    16: 'ட', 17: 'ண', 18: 'த', 19: 'ந', 20: 'ப', 21: 'ம',
    22: 'ய', 23: 'ர', 24: 'ல', 25: 'வ', 26: 'ழ'
}

# Preprocess input word image (noise removal and thresholding)
def preprocess_image(image):
    # Invert and binarize
    _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Morphological operations to remove small noise
    kernel = np.ones((3, 3), np.uint8)
    clean = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    return clean

# Segment characters using contours
def segment_characters(image):
    processed = preprocess_image(image)
    contours, _ = cv2.findContours(processed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Extract and sort bounding boxes left-to-right
    char_regions = [cv2.boundingRect(c) for c in contours]
    char_regions = sorted(char_regions, key=lambda box: box[0])  # Sort by x-coordinate

    characters = []
    for (x, y, w, h) in char_regions:
        # Filter out too-small boxes
        if w < 10 or h < 10:
            continue
        char_img = image[y:y+h, x:x+w]

        # Resize and pad to 64x64
        char_img = resize_and_pad(char_img)
        characters.append(char_img)

    return characters

# Resize image and pad to make it square (64x64)
def resize_and_pad(img):
    h, w = img.shape
    scale = 60.0 / max(h, w)
    img = cv2.resize(img, (int(w * scale), int(h * scale)))
    h, w = img.shape

    # Padding to center the image
    top = (64 - h) // 2
    bottom = 64 - h - top
    left = (64 - w) // 2
    right = 64 - w - left

    padded = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=255)
    return padded

# Predict Tamil word from Brahmi image
def predict_word(image_path):
    word = ""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if image is None:
        print("Error loading image.")
        return

    char_imgs = segment_characters(image)

    if not char_imgs:
        print("No characters found.")
        return

    for char_img in char_imgs:
        norm = char_img / 255.0
        input_img = norm.reshape(1, 64, 64, 1)
        pred = model.predict(input_img)
        label = np.argmax(pred)
        word += label_map.get(label, "?")

    print("Predicted Tamil Word:", word)
    plt.imshow(image, cmap='gray')
    plt.title(f"Input Brahmi Word: {word}")
    plt.axis('off')
    plt.show()

# Example usage
predict_word("/content/drive/MyDrive/Colab Notebooks/TAMIL NLP/datasets/example_word3.png")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'brahmi_to_tamil_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)